# 洗錢
- 筆記: https://www.notion.so/jayschsu/c6a6219dd004469bbbfbecfb6d9883f3

In [542]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import pandas as pd
import numpy as np 
from itertools import product
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from collections import namedtuple

SCORE_TUPLE = namedtuple('SCORE_TUPLE', 'feature, preprocess, sampling, model, iter, precision')

pd.set_option('display.max_rows', 500)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

### Data
- dp_df
    - DEBIT是進錢(+) & CREDIT是出錢(-) 

In [543]:
def display_cust_data(cust_id):
    display(custinfo_df[custinfo_df['cust_id']==cust_id].style.set_caption('custinfo_df'))
    display(ccba_df[ccba_df['cust_id']==cust_id].style.set_caption('ccba_df'))
    display(cdtx_df[cdtx_df['cust_id']==cust_id].style.set_caption('cdtx_df'))
    display(dp_df[dp_df['cust_id']==cust_id].style.set_caption('dp_df'))
    display(remit_df[remit_df['cust_id']==cust_id].style.set_caption('remit_df'))


In [3]:
ccba_df = pd.read_csv('data/dataset1/public_train_x_ccba_full_hashed.csv')
cdtx_df = pd.read_csv('data/dataset1/public_train_x_cdtx0001_full_hashed.csv')
custinfo_df = pd.read_csv('data/dataset1/public_train_x_custinfo_full_hashed.csv')
dp_df = pd.read_csv('data/dataset1/public_train_x_dp_full_hashed.csv')
remit_df = pd.read_csv('data/dataset1/public_train_x_remit1_full_hashed.csv')
tr_alertX_df = pd.read_csv('data/dataset1/train_x_alert_date.csv')
tr_sarY_df = pd.read_csv('data/dataset1/train_y_answer.csv')
public_alertX_df = pd.read_csv('data/dataset1/public_x_alert_date.csv')

**Preprocess**
- 將alert_date, sar_flag merge到custinfo

In [4]:
#mapping alert_date & sar flag
alertDate_dict = tr_alertX_df.set_index('alert_key').to_dict()['date']
sarFlag_dict = tr_sarY_df.set_index('alert_key').to_dict()['sar_flag']
public_alertX_df['sar_flag']=2
alertDatePublc_dict = public_alertX_df.set_index('alert_key').to_dict()['date']
sarFlagPublic_dict = public_alertX_df.set_index('alert_key').to_dict()['sar_flag']
custinfo_df['alert_date'] = custinfo_df['alert_key'].map(lambda x: alertDate_dict[x] if x in alertDate_dict else alertDatePublc_dict[x] )
custinfo_df['sar_flag'] = custinfo_df['alert_key'].map(lambda x: sarFlag_dict[x] if x in sarFlag_dict else sarFlagPublic_dict[x])
custinfo_df['sar_flag_nunique'] = custinfo_df.groupby(['cust_id','alert_date'])['sar_flag'].transform('nunique')
invalid_data_qty = sum(custinfo_df['sar_flag_nunique']>1)
print(f'invalid_data_qty: {invalid_data_qty}')
custinfo_valid = custinfo_df[custinfo_df['sar_flag_nunique']==1]
custinfo_valid.drop_duplicates(['cust_id','alert_date','sar_flag'], inplace=True)
custinfo_valid = custinfo_valid.set_index(['cust_id','alert_date'])


invalid_data_qty: 2


/Users/jayhsu/Library/Python/3.8/lib/python/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


## Feturing

**1. by Date**
1. 把各種資料表轉換為以date為單位的特徵
- ccba: 信用卡相關資料, 以月為單位的資料, 缺少public dataset區間,不需處理
    
- cdtx: 消費細項
    - 每日消費金額
    - 每日消費次數
    - 每日台幣消費金額
    - 每日台幣消費次數   
    - 每日外幣消費金額
    - 每日外幣消費次數
    - 每日國內消費金額
    - 每日國內消費次數    
    - 每日國外消費金額
    - 每日國外消費次數
- dp: 借貸: 金額需要乘上匯率
    - 借貸金額
    - 借貸次數
    - CR金額
    - CR次數
    - DB金額
    - DB次數
    - 臨櫃現金交易金額
    - 臨櫃現金交易次數
    - 非臨櫃現金交易金額
    - 非臨櫃現金交易次數 
    - 跨行交易金額
    - 跨行交易次數
    - 非跨行交易金額
    - 非跨行交易次數
    - 實體ATM交易金額
    - 實體ATM交易次數
    - 非實體ATM交易金額
    - 非實體ATM交易次數  
    - 分行數量
- remit
    - 外匯金額
    - 外匯次數    

In [344]:
#cdtx
cdtxDate_ = cdtx_df.groupby(['cust_id','date']).agg(cdtxAmt_ADate=('amt', sum), cdtxCnt_ADate=('date', 'count'))
cdtxDate_ntd = cdtx_df[cdtx_df['cur_type']==47].groupby(['cust_id','date']).agg(cdtxAmtNTD_ADate=('amt', sum), cdtxCntNTD_ADate=('date', 'count'))
cdtxDate_fc = cdtx_df[cdtx_df['cur_type']!=47].groupby(['cust_id','date']).agg(cdtxAmtFC_ADate=('amt', sum), cdtxCntFC_ADate=('date', 'count'))
cdtxDate_tw = cdtx_df[cdtx_df['country']==130].groupby(['cust_id','date']).agg(cdtxAmtTW_ADate=('amt', sum), cdtxCntTW_ADate=('date', 'count'))
cdtxDate_f = cdtx_df[cdtx_df['country']!=130].groupby(['cust_id','date']).agg(cdtxAmtF_ADate=('amt', sum), cdtxCntF_ADate=('date', 'count'))
cdtxDate_df = pd.concat([cdtxDate_, cdtxDate_ntd, cdtxDate_fc, cdtxDate_tw, cdtxDate_f], axis=1)

#dp
dp_df['amt'] = dp_df['tx_amt']*dp_df['exchg_rate']
dpDate_ = dp_df.groupby(['cust_id','tx_date']).agg(dpAmt_ADate=('tx_amt', sum), dpCnt_ADate=('tx_date', 'count'))
dpDate_CR = dp_df[dp_df['debit_credit']=='CR'].groupby(['cust_id','tx_date']).agg(dpAmtCR_ADate=('tx_amt', sum), dpCntCR_ADate=('tx_date', 'count'))
dpDate_DB = dp_df[dp_df['debit_credit']=='CB'].groupby(['cust_id','tx_date']).agg(dpAmtDB_ADate=('tx_amt', sum), dpCntDB_ADate=('tx_date', 'count'))
dpDate_CC = dp_df[(dp_df['tx_type']==1)&(dp_df['info_asset_code']==12)].groupby(['cust_id','tx_date']).agg(dpAmtCC_ADate=('tx_amt', sum), dpCntCC_ADate=('tx_date', 'count'))
dpDate_NCC = dp_df[~((dp_df['tx_type']==1)&(dp_df['info_asset_code']==12))].groupby(['cust_id','tx_date']).agg(dpAmtNCC_ADate=('tx_amt', sum), dpCntNCC_ADate=('tx_date', 'count'))
dpDate_CBank = dp_df[dp_df['cross_bank']==1].groupby(['cust_id','tx_date']).agg(dpAmtCBank_ADate=('tx_amt', sum), dpCntCBank_ADate=('tx_date', 'count'))
dpDate_InBank = dp_df[dp_df['cross_bank']==0].groupby(['cust_id','tx_date']).agg(dpAmtInBank_ADate=('tx_amt', sum), dpCntInBank_ADate=('tx_date', 'count'))
dpDate_ATM = dp_df[dp_df['ATM']==1].groupby(['cust_id','tx_date']).agg(dpAmtATM_ADate=('tx_amt', sum), dpCntATM_ADate=('tx_date', 'count'))
dpDate_NATM = dp_df[dp_df['ATM']==0].groupby(['cust_id','tx_date']).agg(dpAmtNATM_ADate=('tx_amt', sum), dpCntNATM_ADate=('tx_date', 'count'))
dpDate_branchNunique = dp_df.groupby(['cust_id','tx_date']).agg(dpBranchNunique_ADate=('txbranch', 'nunique'))
dpDate_df = pd.concat([dpDate_, dpDate_CR, dpDate_DB, dpDate_CC, dpDate_NCC, dpDate_CBank, dpDate_InBank, dpDate_ATM, dpDate_NATM, dpDate_branchNunique], axis=1)

#remit
remitDate_df = remit_df.groupby(['cust_id','trans_date']).agg(remitAmt_ADate=('trade_amount_usd', sum), remitCnt_ADate=('trans_no', 'count'))



**2. Accumulate recent K days**   
- accumulate feature: 整合前K天的feature
- Process: 每一個customer要分別做以下的feature engineering
    - 1. 組合cust_id * dates
    - 2. merge with date feature
    - 3. calculate accum feature
    - 4. merge date and accum features


In [349]:
def get_accum_feature(cust_data, cust_id, date_colName, rolling_window):
    dates = list(range(cust_data.index.min(), cust_data.index.max()+1))
    all_custDate = pd.DataFrame(list(product([cust_id], dates)), columns=['cust_id', date_colName]).set_index(['cust_id', date_colName])
    cust_fullDate = pd.merge(cust_data, all_custDate, how='outer', left_index=True, right_index=True)
    cust_fullDate.sort_index(ascending=True, inplace=True)
    cust_fullDate.fillna(0, inplace=True)
    cust_recentAccum = cust_fullDate.rolling(window = rolling_window, min_periods=1).sum().shift()
    new_cols = []
    for c in cust_recentAccum.columns:
        new_cols.append(c+'_Accum')
    cust_recentAccum.columns=new_cols
    feature = pd.merge(cust_data, cust_recentAccum, how='left', left_index=True, right_index=True)
    return feature

def get_accum_feature_V2(cust_data, cust_id, date_colName, rolling_window):
    dates = list(range(cust_data.index.min(), cust_data.index.max()+1))
    all_custDate = pd.DataFrame(list(product([cust_id], dates)), columns=['cust_id', date_colName]).set_index(['cust_id', date_colName])
    cust_fullDate = pd.merge(cust_data, all_custDate, how='outer', left_index=True, right_index=True)
    cust_fullDate.sort_index(ascending=True, inplace=True)
    cust_fullDate.fillna(0, inplace=True)
    
    cust_recentAccum = cust_fullDate.rolling(window=rolling_window, min_periods=1).mean().shift()
    cust_recentAccum2W = cust_fullDate.rolling(window=rolling_window*2, min_periods=1).mean().shift()
    cust_recentAccumDiff = cust_recentAccum*2 - cust_recentAccum2W
    cust_fullDateDiff = cust_fullDate - cust_recentAccum
    
    new_cols = []
    for c in cust_recentAccum.columns:
        new_cols.append(c.replace('_ADate', '_Accum'))
    cust_recentAccum.columns=new_cols
    
    new_cols = []
    for c in cust_recentAccumDiff.columns:
        new_cols.append(c.replace('_ADate', '_AccumDiff'))
    cust_recentAccumDiff.columns=new_cols
    
    new_cols = []
    for c in cust_fullDateDiff.columns:
        new_cols.append(c.replace('_ADate', '_ADateDiff'))
    cust_fullDateDiff.columns=new_cols        
    
    feature = pd.merge(cust_data, cust_fullDateDiff, how='left', left_index=True, right_index=True)
    feature = pd.merge(feature, cust_recentAccum, how='left', left_index=True, right_index=True)
    feature = pd.merge(feature, cust_recentAccumDiff, how='left', left_index=True, right_index=True)
    
    return feature

def accum_featuring_parallel(dateFeature, date_colName='date', rolling_window=5):
    idx_custid, _ = zip(*dateFeature.index)
    idx_custid = np.unique(idx_custid)
    accu_feature_list = Parallel(n_jobs=4)(delayed(get_accum_feature_V2)(dateFeature.loc[(cust_id)], cust_id, date_colName, rolling_window) for cust_id in idx_custid) 
    accu_feature = pd.concat(accu_feature_list)
    return accu_feature

def get_recallN_Precision(y_predProb, y_true):
    y_pred_df = pd.DataFrame(list(zip(y_predProb, y_true)), columns=['predProb','trueLabel'])
    y_pred_df.sort_values(by='predProb', ascending=False, inplace=True)
    y_pred_df['idx']=list(range(len(y_pred_df)))
    idx = y_pred_df[y_pred_df['trueLabel']==1]['idx'].iloc[-2]
    precision_score = sum(y_true)/idx
    return precision_score


In [350]:
%%time
cdtx_feature = accum_featuring_parallel(cdtxDate_df, 'date', rolling_window=3)
dp_feature = accum_featuring_parallel(dpDate_df, 'tx_date', rolling_window=3)
remit_feature = accum_featuring_parallel(remitDate_df, 'trans_date', rolling_window=3)

CPU times: user 33.9 s, sys: 2.63 s, total: 36.5 s
Wall time: 47.3 s


**3. cust profile**

In [452]:
#ccba
ccbaProfile1 = ccba_df.replace(0, np.nan).groupby('cust_id').agg(
                               ccbalupayAmt_Year=('lupay', np.mean),\
                               ccbausgamAmt_Year=('usgam', np.mean),\
                               ccbacycamAax_Year=('cycam', np.max),\
                               ccbaclamtAmt_Year=('clamt', np.mean),\
                               ccbacsamtAmt_Year=('csamt', np.mean),\
                               ccbainamtAmt_Year=('inamt', np.mean),\
                               ccbacucsmAmt_Year=('cucsm', np.mean),\
                               ccbacucahAmt_Year=('cucah', np.mean),\
                              )
ccbaProfile2 = ccba_df.groupby('cust_id').agg(ccbalupayCnt_Year=('lupay', np.count_nonzero),\
                               ccbausgamCnt_Year=('usgam', np.count_nonzero),\
                               ccbaclamtCnt_Year=('clamt', np.count_nonzero),\
                               ccbacsamtCnt_Year=('csamt', np.count_nonzero),\
                               ccbainamtCnt_Year=('inamt', np.count_nonzero),\
                               ccbacucsmCnt_Year=('cucsm', np.count_nonzero),\
                               ccbacucahCnt_Year=('cucah', np.count_nonzero),\
                               ccbabyymmCnt_Year=('byymm', np.count_nonzero),\
                              )
ccbaProfile_df = pd.concat([ccbaProfile1, ccbaProfile2], axis=1)


new_cols = []
for c in custinfo_valid.columns:
    if c in ['alert_key', 'sar_flag', 'sar_flag_nunique']:
        new_cols.append(c)
    else:
        new_cols.append(c+'_Profile')
custinfo_valid.columns=new_cols

custProfile_df = pd.merge(custinfo_valid.reset_index(level=1), ccbaProfile_df, left_index=True, right_index=True, how='left')
custProfile_df.set_index('alert_date', append=True, inplace=True)



**3. Integrate dataset**
- moneyLundry dataset (sar_flag==1, in alert_date)
- falseAlarm dataset (sar_flag==0, in alert_date)
- public dataset (sar_flag==2, in alert_date)

In [453]:
data_df = pd.concat([custProfile_df, cdtx_feature, dp_feature, remit_feature], axis=1)
data_df = data_df[data_df['alert_key'].notna()]

In [459]:
labelY = 'sar_flag'
feature_all = [x for x in data_df.columns if x not in ['alert_key', 'sar_flag', 'sar_flag_nunique']]
feature_Profile = [x for x in data_df.columns if x.split('_')[-1]=='Profile']
feature_ADate = [x for x in data_df.columns if x.split('_')[-1]=='ADate']
feature_ADateDiff = [x for x in data_df.columns if x.split('_')[-1]=='ADateDiff']
feature_Accum = [x for x in data_df.columns if x.split('_')[-1]=='Accum']
feature_AccumDiff = [x for x in data_df.columns if x.split('_')[-1]=='AccumDiff']
feature_Year = [x for x in data_df.columns if x.split('_')[-1]=='Year']
assert len(feature_all)==len(feature_Profile+feature_ADate+feature_ADateDiff+feature_Accum+feature_AccumDiff+feature_Year)

#--- 


display(data_df[labelY].value_counts().to_frame())
dataset_Tr = data_df[data_df['sar_flag']!=2]
dataset_TsPub = data_df[data_df['sar_flag']==2]

print('dataset_Tr:', dataset_Tr.shape)
print('dataset_TsPub:', dataset_TsPub.shape)


,sar_flag
0.0,23451
2.0,1770
1.0,233


dataset_Tr: (23684, 35)
dataset_TsPub: (1770, 35)


## Model Training

In [567]:
pipePreMinMax = Pipeline([('sc', MinMaxScaler()), ('pca', PCA(n_components=0.9))])
pipePreStd = Pipeline([('sc', StandardScaler()), ('pca', PCA(n_components=0.9))])
pipePreMMStd = Pipeline([('sc1', MinMaxScaler()), ('sc2', StandardScaler()), ('pca', PCA(n_components=0.9))])
pipe_list = zip([pipePreMinMax, pipePreStd, pipePreMMStd], ['MinMaxScaler','StandardScaler','MinMaxScaler+StandardScaler'])
pipe_list = list(pipe_list)

pipeSpSmote = imbPipeline([('sp', SMOTE(n_jobs=-1))])
pipeSpBSmote = imbPipeline([('sp', BorderlineSMOTE(n_jobs=-1))])
pipeSpTom = imbPipeline([('sp', TomekLinks(n_jobs=-1))])
pipeSpSmoteTom = imbPipeline([('sp1', BorderlineSMOTE(n_jobs=-1)), ('sp2', TomekLinks(n_jobs=-1))])
sp_list = zip([pipeSpSmote, pipeSpBSmote, pipeSpTom, pipeSpSmoteTom], ['SMOTE', 'BorderlineSMOTE', 'TomekLinks', 'BorderlineSMOTE+TomekLinks'])
sp_list = list(sp_list)

modelLR = LogisticRegression(max_iter=300)
modelXGBC = XGBClassifier(use_label_encoder=False)
modelMLPC = MLPClassifier(max_iter=1000)
model_list = zip([modelLR, modelXGBC, modelMLPC], ['LogisticRegression', 'XGBClassifier', 'MLPClassifier'])
model_list = list(model_list)

featureD = feature_ADate
featurePD = feature_Profile+feature_ADate
featurePDA = feature_Profile+feature_ADate+feature_Accum
featurePDAY = feature_Profile+feature_ADate+feature_Accum+feature_Year
featureD_Diff = feature_ADateDiff
featurePD_Diff = feature_Profile+feature_ADateDiff
featurePDA_Diff = feature_Profile+feature_ADateDiff+feature_AccumDiff
featurePDAY_Diff = feature_Profile+feature_ADateDiff+feature_AccumDiff+feature_Year
featureD_Composite = feature_ADate+feature_ADateDiff
featurePD_Composite = feature_Profile+feature_ADate+feature_ADateDiff
featurePDA_Composite = feature_Profile+feature_ADate+feature_Accum+feature_ADateDiff+feature_AccumDiff
featurePDAY_Composite = feature_Profile+feature_ADate+feature_Accum+feature_ADateDiff+feature_AccumDiff+feature_Year
feature_list = [featureD, featurePD, featurePDA, featurePDAY, featureD_Diff, featurePD_Diff, featurePDA_Diff, featurePDAY_Diff, featureD_Composite, featurePD_Composite, featurePDA_Composite, featurePDAY_Composite]
feature_list = zip(feature_list,['D', 'PD', 'PDA', 'PDAY', 'D_Diff', 'PD_Diff','PDA_Diff','PDAY_Diff','D_Composite','PD_Composite', 'PDA_Composite', 'PDAY_Composite'])
feature_list = list(feature_list)



In [ ]:

for featureX, featureName in feature_list:
    #KFold, Cross-validate
    X = dataset_Tr[featureX].fillna(0).copy()
    Y = dataset_Tr[labelY]
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=23)
    skf.get_n_splits(X,Y)

    score_list = []
    for i, (tr_idx, val_idx) in enumerate(skf.split(X, Y)):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = Y.iloc[tr_idx], Y.iloc[val_idx]

        for pipePre, pipePreName in pipe_list:
            for pipeSp, pipeSpName in sp_list:
                predProb_list = []
                for model, modelName in model_list:
                    #preprocess
                    pipePre.fit(X_tr)
                    x_tr_ = pipePre.transform(X_tr.copy())
                    x_val_ = pipePre.transform(X_val.copy())
                    #over sampling
                    x_re, y_re = pipeSp.fit_resample(x_tr_, y_tr)
                    #tr&pred
                    model = model.fit(x_re, y_re)        
                    y_predProb = model.predict_proba(x_val_)
                    #scole
                    y_predProb = [x[1] for x in y_predProb]
                    precisionScore = get_recallN_Precision(y_predProb, y_val)
                    score_list.append(SCORE_TUPLE(featureName, pipePreName, pipeSpName, modelName, i, precisionScore))
                    predProb_list.append(y_predProb)

                precisionScore = get_recallN_Precision(np.mean(predProb_list, axis=0), y_val)
                score_list.append(SCORE_TUPLE(featureName, pipePreName, pipeSpName, 'ensemble', i, precisionScore))



[19:05:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:07:14] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:08:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:08:28] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[19:30:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:30:53] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:31:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:32:43] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

In [ ]:
score_df = pd.DataFrame(score_list)
display(score_df.groupby('model')['precision'].mean().to_frame())
display(score_df.groupby(['model','iter'])['precision'].max().to_frame())
display(score_df.groupby(['iter','model'])['precision'].max().to_frame())

- profile + date: 0.014338
- date + ccba: 0.012114
- date + accumulate: 0.013388
- date + accumulate + ccbaProfile: 0.012036
- date + dateDiff+ accumulate+ accumulateDiff + ccbaProfile: 0.012238